In [1]:
import time
from selenium import webdriver
from selenium.common import StaleElementReferenceException, TimeoutException, NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains, Keys
from selenium.webdriver.ie.webdriver import WebDriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np

In [2]:
bus_data = {}

bus_route_list = []
bus_route_link_list = []
bus_name_list=[]
bus_type_list = []
departing_time_list = []
duration_list = []
reaching_time_list = []
star_rating_list = []
price_list = []
seats_available_list = []

In [3]:
def get_buses_details(driver):
    
    try:
        next_date = driver.find_element(By.XPATH, "//div[@class='onward d-block fl']//span[@class='next']")
        next_date.click() # next day to get all available buses
        time.sleep(2)
    except:
        pass
    
    try:
        view_gov_buses = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@class='w-14 fl']//div[@class='button']")))

        view_gov_buses.click()
        time.sleep(2)
    except:
        pass
    screen_height = driver.execute_script("return window.screen.height;")
    i=1
    while True:
        driver.execute_script(f"window.scrollTo(0, {screen_height}*{i});")
        i += 1
        time.sleep(1)
        scroll_height = driver.execute_script("return document.body.scrollHeight;")

        if screen_height * i > scroll_height:
            break

    bus_name = driver.find_elements(By.XPATH, "//div[@class = 'travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class = 'bus-type f-12 m-top-16 l-color evBus']")
    departing_time = driver.find_elements(By.XPATH, "//div[@class = 'dp-time f-19 d-color f-bold']")
    duration = driver.find_elements(By.XPATH, "//div[@class = 'dur l-color lh-24']")
    reaching_time = driver.find_elements(By.XPATH, "//div[@class = 'bp-time f-19 d-color disp-Inline']")
    star_rating = driver.find_elements(By.XPATH, "//div[contains(@class, 'rating-sec')]//span[string-length(text()) > 0]")
    price = driver.find_elements(By.XPATH, "//div[@class='seat-fare ']//div[@class='fare d-block']//span")
    seats_available = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for i in range(len(bus_name)):
        bus_name_list.append(bus_name[i].text if i < len(bus_name) else None)
        bus_type_list.append(bus_type[i].text if i < len(bus_type) else None)
        departing_time_list.append(departing_time[i].text if i < len(departing_time) else None)
        duration_list.append(duration[i].text if i < len(duration) else None)
        reaching_time_list.append(reaching_time[i].text if i < len(reaching_time) else None)
        star_rating_list.append(star_rating[i].text if i < len(star_rating) else None)
        price_list.append(price[i].text if i < len(price) else None)
        seats_available_list.append(seats_available[i].text if i < len(seats_available) else None)

    return bus_name

In [4]:
def collect_all_routes(driver):
    all_route_texts = []
    all_route_links = []

    # Fetch all pagination tabs
    pagination_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
    total_page_number = len(pagination_tabs)

    for page_number in range(1, total_page_number + 1):
        # Fetch bus routes for the current page
        bus_route_elements = driver.find_elements(By.XPATH,
                                                  "//div[@class='route_link']//div[@class='route_details']//a[@class='route']")

        if not bus_route_elements:
            print("No bus routes found on this page.")
            break  # Exit if no routes are found

        for route in bus_route_elements:
            all_route_texts.append(route.text)
            all_route_links.append(route.get_attribute("href"))

        # Handle pagination
        try:
            pagination = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[@class='DC_117_paginationTable']"))
            )

            next_page = pagination.find_elements(By.XPATH,
                                                 f"//div[@class='DC_117_pageTabs ' and text()='{page_number + 1}']")
            if next_page:
                # Scroll to the next page element
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page[0])
                time.sleep(1)  # Short wait for scrolling to complete

                # Wait for the element to be clickable
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable(next_page[0]))
                next_page[0].click()  # Click the next page
                time.sleep(2) 
            else:
                print('No more pages')
                break

        except ElementClickInterceptedException:
            print("Element click intercepted. Trying again...")
            time.sleep(1)  
            continue 

        except NoSuchElementException:
            print('No more pages')
            break

    return all_route_texts, all_route_links

In [5]:
def collect_data_from_routes(driver, route_texts, route_links):

    for link in range(len(route_links)):
        driver.get(route_links[link])
        time.sleep(2)

        # Collect bus details
        bus_details = get_buses_details(driver)
        for _ in range(len(bus_details)):
            bus_route_list.append(route_texts[link])
            bus_route_link_list.append(route_links[link])

        driver.back()
        time.sleep(2)

In [6]:
def regional_buses(driver, url):
    driver.get(url)       # https://www.redbus.in/online-booking/rsrtc
    time.sleep(2)
    all_route_texts, all_route_links = collect_all_routes(driver)
    collect_data_from_routes(driver, all_route_texts, all_route_links)

## Rajasthan State Road Transport Corporation 

In [8]:
rsrtc_driver = webdriver.Chrome()
rsrtc_driver.maximize_window()
action = ActionChains(rsrtc_driver)

url = "https://www.redbus.in/online-booking/rsrtc"
regional_buses(rsrtc_driver, url)
rsrtc_driver.close()

No more pages


## Patiala and East Punjab States Union Buses

In [9]:
pepsu_driver = webdriver.Chrome()
pepsu_driver.maximize_window()
action = ActionChains(pepsu_driver)

url = "https://www.redbus.in/online-booking/pepsu"
regional_buses(pepsu_driver, url)
pepsu_driver.close()

No more pages


## Uttar Pradesh State Road Transport Corporation

In [10]:
upsrtc_driver = webdriver.Chrome()
upsrtc_driver.maximize_window()
action = ActionChains(upsrtc_driver)

url = "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc"
regional_buses(upsrtc_driver, url)
upsrtc_driver.close()

No more pages


## Kerala State Road Transport Corporation

In [11]:
ksrtc_driver = webdriver.Chrome()
ksrtc_driver.maximize_window()
action = ActionChains(ksrtc_driver)

url = "https://www.redbus.in/online-booking/ksrtc-kerala"
regional_buses(ksrtc_driver, url)
ksrtc_driver.close()

No more pages


## Meghalaya Transport Corporation(MTC)

In [12]:
mtc_driver = webdriver.Chrome()
mtc_driver.maximize_window()
action = ActionChains(mtc_driver)

url = "https://www.redbus.in/online-booking/meghalaya-transport-corporation-mtc"
regional_buses(mtc_driver, url)
mtc_driver.close()

No more pages


## Telangana State Road Transport Corporation

In [13]:
tsrtc_driver = webdriver.Chrome()
tsrtc_driver.maximize_window()
action = ActionChains(tsrtc_driver)

url = "https://www.redbus.in/online-booking/tsrtc"
regional_buses(tsrtc_driver, url)
tsrtc_driver.close()

No more pages


## Kadamba Transport Corporation Limited (KTCL)

In [14]:
ktcl_driver = webdriver.Chrome()
ktcl_driver.maximize_window()
action = ActionChains(ktcl_driver)

url = "https://www.redbus.in/online-booking/ktcl"
regional_buses(ktcl_driver, url)
ktcl_driver.close()

No more pages


## Chandigarh Transport Undertaking (CTU)

In [15]:
ctu_driver = webdriver.Chrome()
ctu_driver.maximize_window()
action = ActionChains(ctu_driver)

url = "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu"
regional_buses(ctu_driver, url)
ctu_driver.close()

No more pages


## Himachal Road Transport Corporation 

In [16]:
hrtc_driver = webdriver.Chrome()
hrtc_driver.maximize_window()
action = ActionChains(hrtc_driver)

url = "https://www.redbus.in/online-booking/hrtc"
regional_buses(hrtc_driver, url)
hrtc_driver.close()

No more pages


## KAAC Transport

In [17]:
kaac_driver = webdriver.Chrome()
kaac_driver.maximize_window()
action = ActionChains(kaac_driver)

url = "https://www.redbus.in/online-booking/kaac-transport"
regional_buses(kaac_driver, url)
kaac_driver.close()

No more pages


In [18]:
# Get the length of the longest list
max_length = max(len(bus_route_list), len(bus_route_link_list), len(bus_name_list), 
                 len(bus_type_list), len(departing_time_list), len(duration_list), 
                 len(reaching_time_list), len(star_rating_list), len(price_list), 
                 len(seats_available_list))

# Fill shorter lists with NaN
bus_route_list += [np.nan] * (max_length - len(bus_route_list))
bus_route_link_list += [np.nan] * (max_length - len(bus_route_link_list))
bus_name_list += [np.nan] * (max_length - len(bus_name_list))
bus_type_list += [np.nan] * (max_length - len(bus_type_list))
departing_time_list += [np.nan] * (max_length - len(departing_time_list))
duration_list += [np.nan] * (max_length - len(duration_list))
reaching_time_list += [np.nan] * (max_length - len(reaching_time_list))
star_rating_list += [np.nan] * (max_length - len(star_rating_list))
price_list += [np.nan] * (max_length - len(price_list))
seats_available_list += [np.nan] * (max_length - len(seats_available_list))

# Create DataFrame
df = pd.DataFrame({
    'bus_route': bus_route_list,
    'bus_route_link': bus_route_link_list,
    'bus_name': bus_name_list,
    'bus_type': bus_type_list,
    'departing_time': departing_time_list,
    'duration': duration_list,
    'reaching_time': reaching_time_list,
    'star_rating': star_rating_list,
    'price': price_list,
    'seats_available': seats_available_list
})


In [19]:
df

,bus_route,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seats_available
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 194381,Deluxe AC Seater 2+2,06:00,05h 15m,11:15,3.3,527,33 Seats available
1,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 146395,Express Non AC Seater 2+3,06:45,07h 15m,14:00,2.3,299,42 Seats available
2,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 194422,Deluxe AC Seater 2+2,07:15,05h 00m,12:15,3.0,531,22 Seats available
3,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 194255,Express Non AC Seater 2+3,07:45,06h 35m,14:20,5.0,299,46 Seats available
4,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 174533,Express Non AC Seater 2+3,08:45,07h 15m,16:00,4.6,299,42 Seats available
...,...,...,...,...,...,...,...,...,...,...
12053,Guwahati to Manja (assam),https://www.redbus.in/bus-tickets/guwahati-to-...,Kareng Travels,A/C Seater (2+1),09:30,06h 15m,15:45,3.4,400,5 Seats available
12054,Guwahati to Manja (assam),https://www.redbus.in/bus-tickets/guwahati-to-...,Kareng Travels,Non AC Seater (2+2),11:45,06h 15m,18:00,3.9,360,45 Seats available
12055,Hamren to Diphu,https://www.redbus.in/bus-tickets/hamren-to-diphu,KAAC TRANSPORT - 175491,Non AC Seater (2+2),05:30,04h 50m,10:20,3.5,246,41 Seats available
12056,Hamren to Diphu,https://www.redbus.in/bus-tickets/hamren-to-diphu,KAAC TRANSPORT - 157443,Non AC Seater 2+2,11:45,06h 15m,18:00,2.5,300,41 Seats available


In [21]:
df.to_csv('bus_data.csv', index=False)